In [1]:
import numpy

from cicada.additive import AdditiveProtocolSuite
from cicada.communicator import SocketCommunicator
from cicada.encoding import FixedPoint

def main(communicator):
    encoding = FixedPoint(precision=2)
    protocol = AdditiveProtocolSuite(communicator, order=127, encoding=encoding)
    a_share = protocol.share(src=0, secret=numpy.array(2), shape=())
    b_share = protocol.share(src=1, secret=numpy.array(3), shape=())
    c_share = protocol.add(a_share, b_share)
    c = protocol.reveal(c_share)

Note that we're using a very low-precision encoding (`precision=2`) and a very small field (`order=127`) to make the values in the logged output easier to read.

In [2]:
import logging

from cicada import transcript

with transcript.record():
    handler = transcript.net_handler()
    transcript.set_handler(logging.getLogger(), handler)
    SocketCommunicator.run(fn=main, world_size=3);

Player 0: --> 1 PRZS 2113258192961202993
Player 2: --> 0 PRZS 4989078122295468691
Player 1: --> 2 PRZS 1641890102022822472
Player 0: <-- 2 PRZS 4989078122295468691
Player 2: <-- 1 PRZS 1641890102022822472
Player 1: <-- 0 PRZS 2113258192961202993
Player 2: --> 0 GATHER 93
Player 0: --> 0 GATHER 15
Player 1: --> 0 GATHER 39
Player 2: --> 1 GATHER 93
Player 0: <-- 2 GATHER 93
Player 1: --> 1 GATHER 39
Player 2: --> 2 GATHER 93
Player 1: <-- 2 GATHER 93
Player 0: <-- 0 GATHER 15
Player 2: <-- 2 GATHER 93
Player 0: <-- 1 GATHER 39
Player 1: <-- 1 GATHER 39
Player 0: --> 1 GATHER 15
Player 0: --> 2 GATHER 15
Player 1: <-- 0 GATHER 15
Player 2: <-- 0 GATHER 15
Player 1: --> 2 GATHER 39
Player 2: <-- 1 GATHER 39


In [3]:
def main(communicator):
    transcript.log("Let's setup additive sharing!")
    encoding = FixedPoint(precision=2)
    protocol = AdditiveProtocolSuite(communicator, order=127, encoding=encoding)
    transcript.log("Let's share some secrets!")
    a_share = protocol.share(src=0, secret=numpy.array(2), shape=())
    b_share = protocol.share(src=1, secret=numpy.array(3), shape=())
    transcript.log("Let's add some secrets!")
    c_share = protocol.add(a_share, b_share)
    transcript.log("Let's reveal the results!")
    c = protocol.reveal(c_share)

with transcript.record():
    handler = transcript.net_handler()
    transcript.set_handler(logging.getLogger(), handler)
    SocketCommunicator.run(fn=main, world_size=3);

Player 0: Let's setup additive sharing!
Player 2: Let's setup additive sharing!
Player 1: Let's setup additive sharing!
Player 0: --> 1 PRZS 4612915327354008253
Player 1: --> 2 PRZS 7414186504898461884
Player 2: --> 0 PRZS 1882202436348525527
Player 1: <-- 0 PRZS 4612915327354008253
Player 2: <-- 1 PRZS 7414186504898461884
Player 0: <-- 2 PRZS 1882202436348525527
Player 1: Let's share some secrets!
Player 2: Let's share some secrets!
Player 0: Let's share some secrets!
Player 1: Let's add some secrets!
Player 2: Let's add some secrets!
Player 0: Let's add some secrets!
Player 1: Let's reveal the results!
Player 2: Let's reveal the results!
Player 0: Let's reveal the results!
Player 1: --> 0 GATHER 6
Player 2: --> 0 GATHER 75
Player 0: --> 0 GATHER 66
Player 1: --> 1 GATHER 6
Player 2: --> 1 GATHER 75
Player 0: <-- 1 GATHER 6
Player 2: --> 2 GATHER 75
Player 1: <-- 1 GATHER 6
Player 0: <-- 2 GATHER 75
Player 2: <-- 2 GATHER 75
Player 1: <-- 2 GATHER 75
Player 0: <-- 0 GATHER 66
Player 0

In [4]:
with transcript.record():
    fmt = "# {processName}: {msg}"
    netfmt = "{processName},{net.verb},{net.src},{net.dst},{net.tag},{net.payload}"
    handler = transcript.net_handler(fmt=fmt, netfmt=netfmt)
    transcript.set_handler(logging.getLogger(), handler)
    SocketCommunicator.run(fn=main, world_size=3);

# Player 0: Let's setup additive sharing!
# Player 1: Let's setup additive sharing!
# Player 2: Let's setup additive sharing!
Player 0,sent,0,1,PRZS,6501041567845331950
Player 2,sent,2,0,PRZS,6064374732292475082
Player 1,sent,1,2,PRZS,3210179197484905169
Player 1,received,0,1,PRZS,6501041567845331950
Player 0,received,2,0,PRZS,6064374732292475082
Player 2,received,1,2,PRZS,3210179197484905169
# Player 1: Let's share some secrets!
# Player 0: Let's share some secrets!
# Player 2: Let's share some secrets!
# Player 1: Let's add some secrets!
# Player 2: Let's add some secrets!
# Player 0: Let's add some secrets!
# Player 1: Let's reveal the results!
# Player 2: Let's reveal the results!
# Player 0: Let's reveal the results!
Player 1,sent,1,0,GATHER,95
Player 2,sent,2,0,GATHER,12
Player 0,sent,0,0,GATHER,40
Player 1,sent,1,1,GATHER,95
Player 2,sent,2,1,GATHER,12
Player 0,received,1,0,GATHER,95
Player 1,received,1,1,GATHER,95
Player 0,received,2,0,GATHER,12
Player 2,sent,2,2,GATHER,12
Play

If you look carefully, you can see that this produces two nearly identical events for each message (once when the message is sent, and once when the message is received).  If you wish to eliminate the duplication, e.g. by only logging messages when they're sent, you can specify that too:

In [5]:
with transcript.record():
    fmt = "# {processName}: {msg}"
    netfmt = "{processName},{net.verb},{net.src},{net.dst},{net.tag},{net.payload}"
    handler = transcript.net_handler(received=False, fmt=fmt, netfmt=netfmt)
    transcript.set_handler(logging.getLogger(), handler)
    SocketCommunicator.run(fn=main, world_size=3);

# Player 0: Let's setup additive sharing!
# Player 1: Let's setup additive sharing!
# Player 2: Let's setup additive sharing!
Player 0,sent,0,1,PRZS,4923441390019161378
Player 2,sent,2,0,PRZS,3792132382029304949
Player 1,sent,1,2,PRZS,8009281797746407522
# Player 0: Let's share some secrets!
# Player 1: Let's share some secrets!
# Player 2: Let's share some secrets!
# Player 0: Let's add some secrets!
# Player 2: Let's add some secrets!
# Player 1: Let's add some secrets!
# Player 0: Let's reveal the results!
# Player 2: Let's reveal the results!
# Player 1: Let's reveal the results!
Player 0,sent,0,0,GATHER,99
Player 2,sent,2,0,GATHER,64
Player 1,sent,1,0,GATHER,111
Player 2,sent,2,1,GATHER,64
Player 1,sent,1,1,GATHER,111
Player 0,sent,0,1,GATHER,99
Player 2,sent,2,2,GATHER,64
Player 0,sent,0,2,GATHER,99
Player 1,sent,1,2,GATHER,111


In [6]:
from cicada.arithmetic import Field

with transcript.record():
    handler = transcript.code_handler()
    transcript.set_handler(logging.getLogger(), handler)
    
    f = Field(order=127)
    a = f.ones(3)
    b = f.uniform(size=3, generator=numpy.random.default_rng())
    f.inplace_add(a, b)

cicada.transcript.assert_equal(cicada.arithmetic.Field(order=127).ones(shape=3), numpy.array([1, 1, 1], dtype='object'))

bg = numpy.random.PCG64()
bg.state = {'bit_generator': 'PCG64', 'state': {'state': 310581584614395710612860853258973098112, 'inc': 87454387631014545490430812487103061493}, 'has_uint32': 0, 'uinteger': 0}
cicada.transcript.assert_equal(cicada.arithmetic.Field(order=127).uniform(size=3, generator=numpy.random.Generator(bg)), numpy.array([24, 47, 119], dtype='object'))

lhs = numpy.array([1, 1, 1], dtype='object')
cicada.arithmetic.Field(order=127).inplace_add(lhs=lhs, rhs=numpy.array([24, 47, 119], dtype='object'))
cicada.transcript.assert_equal(lhs, numpy.array([25, 48, 120], dtype='object'))



In [7]:
import io

with transcript.record():
    buffer = io.StringIO()
    handler = transcript.code_handler(logging.StreamHandler(buffer))
    transcript.set_handler(logging.getLogger(), handler)
    
    f = Field(order=127)
    a = f.ones(3)
    b = f.uniform(size=3, generator=numpy.random.default_rng())
    f.inplace_add(a, b)

In [8]:
buffer.seek(0)
for line in buffer:
    print(line.strip())

cicada.transcript.assert_equal(cicada.arithmetic.Field(order=127).ones(shape=3), numpy.array([1, 1, 1], dtype='object'))

bg = numpy.random.PCG64()
bg.state = {'bit_generator': 'PCG64', 'state': {'state': 294111206801531245155334061750484527776, 'inc': 44482354329879700285959243340315682281}, 'has_uint32': 0, 'uinteger': 0}
cicada.transcript.assert_equal(cicada.arithmetic.Field(order=127).uniform(size=3, generator=numpy.random.Generator(bg)), numpy.array([42, 25, 0], dtype='object'))

lhs = numpy.array([1, 1, 1], dtype='object')
cicada.arithmetic.Field(order=127).inplace_add(lhs=lhs, rhs=numpy.array([42, 25, 0], dtype='object'))
cicada.transcript.assert_equal(lhs, numpy.array([43, 26, 1], dtype='object'))



Manual inspection shows that the transcript contains the expected output. Now, we can execute the buffered transcript, one-line-at-a-time, and confirm that none of the assertions raise exceptions:

In [9]:
import cicada.transcript

try:
    buffer.seek(0)
    for line in buffer:
        exec(line)
except Exception as e:
    print(f"Consistency verification failed: {e}")
else:
    print("Consistency verification succeeded.")

Consistency verification succeeded.


Let's go back to our original example. Because it requires three players, we'll setup Cicada to write a separate transcript for each player:  

In [10]:
def main(comm):
    handler = transcript.code_handler(logging.FileHandler(f"player-{comm.rank}.py", "w"))
    transcript.set_handler(logging.getLogger(), handler)

    transcript.log("Let's setup additive sharing!")
    encoding = FixedPoint(precision=2)
    protocol = AdditiveProtocolSuite(comm, order=127, encoding=encoding)
    transcript.log("Let's share some secrets!")
    a_share = protocol.share(src=0, secret=numpy.array(2), shape=())
    b_share = protocol.share(src=1, secret=numpy.array(3), shape=())
    transcript.log("Let's add some secrets!")
    c_share = protocol.add(a_share, b_share)
    transcript.log("Let's reveal the results!")
    c = protocol.reveal(c_share)

with transcript.record():
    SocketCommunicator.run(fn=main, world_size=3);

Note that we've moved our transcript configuration code inside `main`, which allows us to generate a separate filename for each transcript based on player rank. Also note that we've left our context messages in-place. Let's see what the results look like for one of the players:

In [11]:
with open("player-0.py") as stream:
    for line in stream:
        print(line.strip())

# Let's setup additive sharing!
# Let's share some secrets!
# cicada.additive.AdditiveProtocolSuite().share(src=0, secret=numpy.array(2, dtype='int64'), shape=(), encoding=None)

bg = numpy.random.PCG64()
bg.state = {'bit_generator': 'PCG64', 'state': {'state': 275934374581578202693231533907928662192, 'inc': 74303973817490204264452453842712767295}, 'has_uint32': 0, 'uinteger': 0}
cicada.transcript.assert_equal(cicada.arithmetic.Field(order=127).uniform(size=(), generator=numpy.random.Generator(bg)), numpy.array(62, dtype='object'))

bg = numpy.random.PCG64()
bg.state = {'bit_generator': 'PCG64', 'state': {'state': 45413300204548699694289800305975195387, 'inc': 122458647806018768901565485422780565793}, 'has_uint32': 0, 'uinteger': 0}
cicada.transcript.assert_equal(cicada.arithmetic.Field(order=127).uniform(size=(), generator=numpy.random.Generator(bg)), numpy.array(117, dtype='object'))

lhs = numpy.array(62, dtype='object')
cicada.arithmetic.Field(order=127).inplace_subtract(lhs=lhs, r

In [12]:
import traceback

for rank in range(3):
    try:
        with open(f"player-{rank}.py") as stream:
            for line in stream:
                exec(line)
    except Exception as e:
        print(f"Player {rank} consistency verification failed: {e}")
        print(traceback.format_exc())
    else:
        print(f"Player {rank} consistency verification succeeded.")

Player 0 consistency verification succeeded.
Player 1 consistency verification succeeded.
Player 2 consistency verification succeeded.


In [13]:
def main(comm):
    handler = logging.StreamHandler() if comm.rank == 0 else logging.NullHandler()
    handler = transcript.code_handler(handler, sent=True, received=True)
    transcript.set_handler(logging.getLogger(), handler)

    transcript.log("Let's setup additive sharing!")
    encoding = FixedPoint(precision=2)
    protocol = AdditiveProtocolSuite(comm, order=127, encoding=encoding)
    transcript.log("Let's share some secrets!")
    a_share = protocol.share(src=0, secret=numpy.array(2), shape=())
    b_share = protocol.share(src=1, secret=numpy.array(3), shape=())
    transcript.log("Let's add some secrets!")
    c_share = protocol.add(a_share, b_share)
    transcript.log("Let's reveal the results!")
    c = protocol.reveal(c_share)

with transcript.record():
    SocketCommunicator.run(fn=main, world_size=3);

# Let's setup additive sharing!
# 0 <-- 2 PRZS 2484391363636162676
# 0 --> 1 PRZS 3705543425192515578
# 0 <-- 2 GATHER 11
# Let's share some secrets!
# cicada.additive.AdditiveProtocolSuite().share(src=0, secret=numpy.array(2, dtype='int64'), shape=(), encoding=None)

# 0 <-- 1 GATHER 95
bg = numpy.random.PCG64()
bg.state = {'bit_generator': 'PCG64', 'state': {'state': 70824561701714392247314131268118952641, 'inc': 248539783406888609260376891470122436953}, 'has_uint32': 0, 'uinteger': 0}
cicada.transcript.assert_equal(cicada.arithmetic.Field(order=127).uniform(size=(), generator=numpy.random.Generator(bg)), numpy.array(31, dtype='object'))

bg = numpy.random.PCG64()
bg.state = {'bit_generator': 'PCG64', 'state': {'state': 327741436583401028878067909547269207792, 'inc': 66951744883520492504918500539108410445}, 'has_uint32': 0, 'uinteger': 0}
cicada.transcript.assert_equal(cicada.arithmetic.Field(order=127).uniform(size=(), generator=numpy.random.Generator(bg)), numpy.array(43, dtype='ob